<a href="https://colab.research.google.com/github/MarianickB/BMP/blob/master/DecisionTreeCode_MiniProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np 
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

class Node:
  def __init__(self, data_indices, parent):
    self.data_indices = data_indices
    self.left = None
    self.right = None
    self.split_feature = None
    self.split_value = None

    if parent:
      self.depth = parent.depth+1
      self.num_classes = parent.num_classes
      self.data = parent.data
      self.labels = parent.labels
      class_prob = np.bincount(self.labels[data_indices], minlength = self.num_classes)
      self.class_prob = class_prob/np.sum(class_prob)

def greedy(node, cost):
  best_cost = np.inf
  best_feature = None
  best_value = None
  num_instances = node.data.shape
  num_features = node.data.shape

  data_sorted = np.sort(node.data[node.data_indices], axis=0)
  test_candidates = (data_sorted[1:]+data_sorted[:-1])/2

  for f in range(num_features):
    data_f = node.data[node.data_indices, f]
    for test in test_candidates[:,f]:
      left_indices = node.data_indices[data_f<=test]
      right_indices = node.data_indices[data_f>test]

      if len(right_indices)==0 or len(left_indices)==0:
        continue
      left_cost = cost(node.labels[left_indices])
      right_cost = cost(node.labels[right_indices])
      
      num_left = left_indices.shape[0]
      num_right = right_indices.shape[0]

      totcost = (num_left*left_cost + num_right*right_cost)/num_instances

      if totcost<best_cost:
        best_cost = totcost
        best_feature = f
        best_value = test
  return best_cost, best_feature, best_value


def misclassification_cost(labels):
  counts = np.bincount(labels)
  class_probs=counts/np.sum(counts)
  return 1-np.max(class_probs)

def cost_entropy(labels):
  class_probs = np.bincounts(labels)/len(labels)
  class_probs = class_probs[class_probs>0]
  return -np.sum(class_probs*np.log(class_probs))

def gini_index(labels):
  class_probs = np.bicounts(labels)/len(labels)
  return 1-np.sum(np.square(class_probs))

class DecisionTree:
  def __init__(self, num_classes = None, max_depth=3, cost = misclassification_cost, min_leaf_instances = 1):
    self.max_depth = max_depth
    self.root = None
    self.cost = cost
    self.num_classes = num_classes
    self.min_leaf_instances = min_leaf_instances

  def fit(self, data, labels):
    pass
  
  def predict(self, data_test):
    pass

def fit(self, data, labels):
  self.data = data
  self.labels = labels

  if self.num_classes is None: 
    self.num_classes = np.max(labels)+1

  self.root = Node(np.arange(data.shape[0]), None)
  self.root.data = data
  self.root.labels = labels
  self.root.num_classes = self.num_classes
  self.root.depth = 0
  self._fit_tree(self.root)
  return self

def _fit_tree(self, node):
  if node.depth == self.max_depth or len(node.data_indices) <= self.min_leaf_instances:
    return
  
  cost, split_feature, split_value = greedy(node, self.cost)
  
  if np.isinf(cost):
    return
  
  test = node.data[node.data_indices, split_feature] <=split_value
  node.split_feature = split_feature
  node.split_value = split_value

  left = Node(node.data_indices[test], node)
  right = Node(node.data_indices[np.logical_not(test)], node)
  self._fit_tree(left)
  self._fit_tree(right)

DecisionTree.fit = fit
DecisionTree._fit_tree = _fit_tree


def predict(self, data_test):
  class_probs = np.zeros((data_test.shape[0], self.num_classes))
  for n, x in enumerate(data_test):
    node  = self.root

    while node.left:
      if x[node.split_feature] <=node.split_value:
        node = node.left
      else:
        node = node.right

    class_probs[n,:] = node.class_prob
  return class_probs

DecisionTree.predict = predict



# Experiments for Decision Trees

In [15]:
from sklearn import datasets
dataset = datasets.load_iris()
x,y = dataset['data'][:,:2], dataset['target']
(num_instances, num_features), num_classes = x.shape, np.max(y)+1
indx = np.random.permutation(num_instances)

x_train,  y_train = x[indx[:200]], y[indx[:200]]

x_test,  y_test= x[indx[200:]], y[indx[200:]]


tree = DecisionTree(max_depth=30)
probs_test = tree.fit(x_train, y_train).predict(x_test)

y_pred = np.argmax(probs_test,1)
accuracy = np.sum(y_pred == y_test)/y_test.shape[0]
print(f'Tree accuracy: {accuracy*100:1f}.')

correct = y_test == y_pred
incorrect = np.logical_not(correct)

plt.scatter(x_train[:,0], x_train[:, 1], c=y_train, marker='o', alpha=.2, label='trainset')
plt.scatter(x_test[correct, 0], x_test[correct, 1], marker='.', c=y_pred[correct], label='correct')
plt.scatter(x_test[incorrect, 0], x_test[incorrect, 1], marker='x', c=y_test[incorrect], label='incorrect')
plt.legend()
plt.show()

TypeError: ignored